<a href="https://colab.research.google.com/github/KakashiDin/coursera-capstone/blob/master/airbnb_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tomslee_airbnb_boston_0282_2016-01-16.csv to tomslee_airbnb_boston_0282_2016-01-16.csv


In [ ]:
import io
bjan = pd.read_csv(io.BytesIO(uploaded['tomslee_airbnb_boston_0282_2016-01-16.csv']))

In [ ]:
bjan.head()

,room_id,host_id,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,minstay,latitude,longitude,last_modified
0,7275108,892781.0,Entire home/apt,NaN,South End,3,5.0,NaN,2.0,380.0,2.0,42.343598,-71.071908,2016-01-16 14:20:07.808024
1,613719,970426.0,Entire home/apt,NaN,Beacon Hill,12,4.5,4.0,1.0,180.0,7.0,42.358973,-71.067036,2016-01-16 14:20:01.317710
2,1724900,7723912.0,Entire home/apt,NaN,Roslindale,18,4.5,4.0,2.0,146.0,5.0,42.282905,-71.132677,2016-01-16 14:19:51.321258
3,3727560,19036819.0,Private room,NaN,Roxbury,76,4.5,2.0,1.0,70.0,2.0,42.329231,-71.079758,2016-01-16 14:19:41.577866
4,8309604,4127253.0,Entire home/apt,NaN,Back Bay,1,5.0,4.0,2.0,290.0,2.0,42.348327,-71.073732,2016-01-16 14:19:26.414634


when taking dot product we were getting NaN as output. Now checking for null values in minstay and reviews. In minstay we found around 80 missing values. After removing them we got a scalar, but we wanted a list. Lets deal with it!

In [ ]:
d=pd.isna(bjan['minstay'])
e=pd.DataFrame(bjan.loc[lambda bjan: d==True])
f=e.index.tolist()
bjan.drop(index=f,inplace=True)

l=pd.isna(bjan['accommodates'])
m=pd.DataFrame(bjan.loc[lambda bjan: l==True])
o=m.index.tolist()
bjan.drop(index=o,inplace=True)
bjan.loc[lambda bjan: l==True] #checking

,room_id,host_id,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,minstay,latitude,longitude,last_modified


In [ ]:
g=bjan[['reviews']]
h=bjan[['minstay']]
i=bjan[['accommodates']]
h.rename(columns={'minstay':'flux'},inplace=True)
g.rename(columns={'reviews':'flux'},inplace=True)
i.rename(columns={'accommodates':'flux'},inplace=True)
j=g*h*i
j



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,flux
1,336.0
2,360.0
3,304.0
4,8.0
5,0.0
...,...
2822,192.0
2823,138.0
2824,0.0
2825,12.0


So finally we were able to get the flux, a list of products between reviews and minstay. Lets involve accomodates as well now.

In [ ]:
bflux=[bjan,j]
bjf=pd.concat(bflux, axis=1)

In [ ]:
j.head()

,flux
1,336.0
2,360.0
3,304.0
4,8.0
5,0.0


In [ ]:
bjf.head()

,room_id,host_id,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,minstay,latitude,longitude,last_modified,flux
1,613719,970426.0,Entire home/apt,NaN,Beacon Hill,12,4.5,4.0,1.0,180.0,7.0,42.358973,-71.067036,2016-01-16 14:20:01.317710,336.0
2,1724900,7723912.0,Entire home/apt,NaN,Roslindale,18,4.5,4.0,2.0,146.0,5.0,42.282905,-71.132677,2016-01-16 14:19:51.321258,360.0
3,3727560,19036819.0,Private room,NaN,Roxbury,76,4.5,2.0,1.0,70.0,2.0,42.329231,-71.079758,2016-01-16 14:19:41.577866,304.0
4,8309604,4127253.0,Entire home/apt,NaN,Back Bay,1,5.0,4.0,2.0,290.0,2.0,42.348327,-71.073732,2016-01-16 14:19:26.414634,8.0
5,8761060,45942466.0,Private room,NaN,Mattapan,0,NaN,3.0,1.0,100.0,1.0,42.276161,-71.075306,2016-01-16 14:19:17.299072,0.0


Perfect! we have our required flux. Moreover, there is no NaN value in flux column. Lets make heat map now in folium maps.

In [ ]:
bjf1=bjf[['room_id','host_id','neighborhood','flux','price','latitude','longitude']]

In [ ]:
bjf1.head()

,room_id,host_id,neighborhood,flux,price,latitude,longitude
1,613719,970426.0,Beacon Hill,336.0,180.0,42.358973,-71.067036
2,1724900,7723912.0,Roslindale,360.0,146.0,42.282905,-71.132677
3,3727560,19036819.0,Roxbury,304.0,70.0,42.329231,-71.079758
4,8309604,4127253.0,Back Bay,8.0,290.0,42.348327,-71.073732
5,8761060,45942466.0,Mattapan,0.0,100.0,42.276161,-71.075306


In [ ]:
import plotly.express as px
fig = px.density_mapbox(bjf1, lat='latitude', lon='longitude', z='flux', radius=13,
                        center=dict(lat=42.340674 , lon=-71.082529), zoom=11,
                        mapbox_style="stamen-terrain")
fig.show()
